In [24]:
# Load Libraries
import pandas as pd
import numpy as np
import re
import datetime

In [25]:
# Load excel file into pandas dataframe
df = pd.read_excel("PackageWiseAdmission.xlsx", index_col=None)
df.dropna(subset=['Course Package'], inplace=True)
df = df[(df['Lead Stage'] != 'Dropped Out') & (df['Admission Status'] != 'Dropped Out')]

In [26]:
# Inspect Dataframe
df.columns
df.dtypes
df.shape
df.isnull().sum()

Course Package                       0
Academic Year                        0
Inquiry Date                         0
Admission Batch                      1
Training Program Funding Source      5
Start Date                           0
Expected End Date                    0
Invoice #                            0
Admission Date                       0
Student Name                         0
Lead Source                          0
Counsellor                           0
Counselor email                     81
Admission Status                     0
State                                0
Status Date                          0
Lead Stage                           0
Package Amount                       8
Comments                           187
Placement Salary Range             204
Placement Job Title                160
Placement Information               87
Placement Employer                 154
Placement Remark                   107
dtype: int64

In [27]:
df["Course Package"] = df["Course Package"].apply(lambda x: x.split('_', 1)[1] if '_' in str(x) else x)
df["Course Package"] = df["Course Package"].apply(lambda x: x.split('_Pack')[0] if '_Package' in str(x) else x)
df["Course Package"] = df["Course Package"].apply(lambda x: x.split('_', 1)[1] if '_' in str(x) else x)
df["Lead Source"] = df["Lead Source"].apply(lambda x: x.split('_')[1] if '_' in str(x) else x)

In [28]:
date_columns = ['Inquiry Date','Start Date','Expected End Date']
df[date_columns] = df[date_columns].apply(pd.to_datetime)

In [29]:
df["Placement Information"].isnull().sum()
df['Placement Information'] = df['Placement Information'].apply(lambda x: 1 if x == 'Yes' else 0)

In [30]:
df = df[~(df['Expected End Date'] >= pd.Timestamp(datetime.date(2023, 5, 30)))]

In [31]:
end_2019 = pd.Timestamp(datetime.date(2019, 6, 30))
end_2020 = pd.Timestamp(datetime.date(2020, 6, 30))
end_2021 = pd.Timestamp(datetime.date(2021, 6, 30))
end_2022 = pd.Timestamp(datetime.date(2022, 6, 30))
end_2023 = pd.Timestamp(datetime.date(2023, 5, 30))

In [32]:
def performance_year(date):
    if date <= end_2019:
        return '2018'
    elif date <= end_2020:
        return '2019'
    elif date <= end_2021:
        return '2020'
    elif date <= end_2022:
        return '2021'
    elif date <= end_2023:
        return '2022'
    else:
        return '2023'

In [33]:
df['Performance Year'] = df['Start Date'].apply(performance_year)

In [35]:
df.to_excel('Current Year Performance_1.xlsx', index=False)

In [34]:
#df[df['Expected End Date'] >= pd.Timestamp(datetime.date(2023, 5, 30))]
len(df['Performance Year'][df['Expected End Date'] >= pd.Timestamp(datetime.date(2023, 5, 30))])

0